In [1]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os

# Import constants.py and necessary functions from bit and web3
from bit import *
from web3 import Web3
from constants import *
from pathlib import Path
from getpass import getpass
from dotenv import load_dotenv
from eth_account import Account 
from bit.network import NetworkAPI
from web3 import Web3, HTTPProvider
from web3.middleware import geth_poa_middleware
from bit import Key, PrivateKey, PrivateKeyTestnet




In [2]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [3]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

In [4]:
# Create a function called `derive_wallets`
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)


In [5]:
from pprint import pprint 

In [6]:
# Creates a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

pprint(coins)

{'btc-test': [{'address': 'mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD',
               'index': 0,
               'path': "m/44'/1'/0'/0/0",
               'privkey': 'cUMT7ptJgxrwSm9s5zSUCKUhSNVq1UC9v31vgoSusyukAciNMdDt',
               'pubkey': '02dcb7a11c7172966e58679c0cb207bfb95d119cc55f9fb318d0a522aaff5b9577',
               'pubkeyhash': '3b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb8',
               'xprv': 'tprv8khhNkzGYHzyhzgCwAKEf4xVvkBbV2vWqBQpybEMhGHivv3WgwhnjdCguDwQ1ujRnN5JWEM8q8nE8zjHSrthSTvMcxb6V575s5x2zT3q1Pk',
               'xpub': 'tpubDHPjXB2WgfgebThzpoyq4UccVmhXeN7RQV1cG7Gf7Y67mQJHKLXNv7pZ5M2WvzFPpQyVgMB9znNEFMefiFvgn7r72wMott5xBJksEZUfWX5'},
              {'address': 'mmUQuAvBYqVpKzayhaYvnJe9VuwkRdSWPu',
               'index': 1,
               'path': "m/44'/1'/0'/0/1",
               'privkey': 'cVj1Q51vqgtBEYokpcb6RtjMwVbAsaCmrQRyjMTHFm5rGoziYe44',
               'pubkey': '024fca815b769b3e6920eb31081de336536e0a0dcba4955b6d7773b8acc205e0b4',
               'pubkeyhash': '

In [7]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.

def priv_key_to_account(coin, priv_key):
    if coin == 'BTCTEST':
        return PrivateKeyTestnet(priv_key)
    elif coin == 'ETH':
        return Account.privateKeyToAccount(priv_key)


In [8]:
eth_PrivateKey = coins["eth"][0]['privkey']
btc_PrivateKey = coins['btc-test'][1]['privkey']
print(eth_PrivateKey)
print(btc_PrivateKey)

0xd36d0f9ff2f6dc80208cd19e0b7d939f11bd1a63d10c38cdc1c58f4dbbc95edc
cVj1Q51vqgtBEYokpcb6RtjMwVbAsaCmrQRyjMTHFm5rGoziYe44


In [9]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)
print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: mmUQuAvBYqVpKzayhaYvnJe9VuwkRdSWPu>


In [10]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient, amount):
    global tx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return tx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)]) 


In [11]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    if coin == "eth": 
        trx_eth = create_tx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_tx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

In [12]:
# create BTC transaction
print(create_tx(BTCTEST,btc_acc,"mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD", 0.000369))

{"unspents":[{"amount":456884,"confirmations":8,"script":"76a9144155b748ac4f668e327a8b7a7c928241ba09eb8588ac","txid":"98179357d7bbdac3eac47237f266693c6f514b137eea3b187338d4f31bdf760a","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD",36900],["mmUQuAvBYqVpKzayhaYvnJe9VuwkRdSWPu",396932]]}


In [13]:
# send BTC transaction
print(send_tx(BTCTEST,btc_acc,'mkwqRYLAfeoMJnnb5A9WegujdggyBeEPGD',0.000369))


01000000010a76df1bf3d43873183bea7e134b516f3c6966f23772c4eac3dabbd757931798010000006a47304402206ec5286ef6cf612acb4c72b0f0646985c880d54eebeb8fe982cd7f07d6aa713c02205395d92a56f16318bf9b4c0e2702060dc12eb25bf302aa986154165142cccabd0121024fca815b769b3e6920eb31081de336536e0a0dcba4955b6d7773b8acc205e0b4ffffffff0224900000000000001976a9143b8d4adb98c1922686a0b9b3f10ee0f1ef7a2eb888ac840e0600000000001976a9144155b748ac4f668e327a8b7a7c928241ba09eb8588ac00000000


In [14]:
# ETH transactions
#check  balance of the account
w3.eth.getBalance("0x02194a55DDA1B029F70220D53f099F38c629ab36")




45000000000000000000

In [20]:
# create eth transaction
create_tx(ETH,eth_acc,"0x3dFb64A5AdAe2aDece2E252FE5E889d61Ef75122", 25000000000000000000)


{'to': '0x3dFb64A5AdAe2aDece2E252FE5E889d61Ef75122',
 'from': '0x02194a55DDA1B029F70220D53f099F38c629ab36',
 'value': 25000000000000000000,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 2}

In [21]:
# send eth transaction
send_tx(ETH, eth_acc,"0x3dFb64A5AdAe2aDece2E252FE5E889d61Ef75122", 25000000000000000000)

0xd0be5c8c26b49872f9aaec8becbf359e47580901e9707bf6a4071e17a5cf6bf3


'0xd0be5c8c26b49872f9aaec8becbf359e47580901e9707bf6a4071e17a5cf6bf3'